In [1]:
import os

os.nice(1)

import numpy as np
import re
import dill
import glob

from rasterio.plot import reshape_as_raster
from rasterio.windows import get_data_window

import imageio
import gc
from rasterio.windows import Window
import rasterio

In [2]:
dill.load_session('/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/Model/Model_Oesterreich4.db')

In [3]:
path_s2 = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/'
path_S1_VV = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/resamp_VV/'
path_S1_VH = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/resamp_VH/'
path_dgm = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/dgm/'
path_slope = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/slope/'

In [4]:
tile = '32TPT'
filenames_S2 = glob.glob(path_s2+'*'+tile+'*.tif')
filenames_S2

filenames_S1_VV = glob.glob(path_S1_VV+'*'+tile+'*.tif')
filenames_S1_VV

filenames_S1_VH = glob.glob(path_S1_VH+'*'+tile+'*.tif')
filenames_S1_VH

filenames_dgm = glob.glob(path_dgm+'*'+tile+'*.tif')
filenames_dgm

filenames_slope = glob.glob(path_slope+'*'+tile+'*.tif')
filenames_slope


images = filenames_S2 + filenames_dgm + filenames_slope + filenames_S1_VV + filenames_S1_VH
images

['/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-03-03.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-03-18.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-04-02.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-04-17.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-05-02.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-05-17.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-06-01.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-06-16.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-07-01.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-07-16.tif',
 '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/S22neu/cube_32TPT_B02_2019-07-31.tif',
 '/mnt/eo/projekt/2021_STATeo/01

In [6]:
sample_S2 = imageio.imread(filenames_S2[0])
sample_S2 = sample_S2[:,:,np.newaxis]
x,y,z = sample_S2.shape


In [9]:
fit.classes_

array(['Bare rock and screes', 'Cropland', 'Feuchtgebiete', 'Gletscher',
       'Grünland', 'Heathland and shrubs', 'Urban', 'Wasser', 'Wälder',
       'sparsely vegetated land'], dtype=object)

In [10]:
list_tiles = ['32TNS','32TNT','32TPS','32TPT', '32TQS', '32TQT', '32UQU', '33TUM', '33TUN', '33TVM', '33TVN', '33TWM', '33TWN', '33TXN', '33UUP', '33UVP', '33UVQ', '33UWP', '33UWQ', '33UXP', '33UXQ']

In [11]:
for t in list_tiles:
    tile = t
    filenames_S2 = glob.glob(path_s2+'*'+tile+'*.tif')
    filenames_S2

    filenames_S1_VV = glob.glob(path_S1_VV+'*'+tile+'*.tif')
    filenames_S1_VV

    filenames_S1_VH = glob.glob(path_S1_VH+'*'+tile+'*.tif')
    filenames_S1_VH

    filenames_dgm = glob.glob(path_dgm+'*'+tile+'*.tif')
    filenames_dgm

    filenames_slope = glob.glob(path_slope+'*'+tile+'*.tif')
    filenames_slope

    images = filenames_S2 + filenames_dgm + filenames_slope+ filenames_S1_VV + filenames_S1_VH
    images
    
    tile_name = [tile]

    sample_S2 = imageio.imread(filenames_S2[0])
    sample_S2 = sample_S2[:,:,np.newaxis]
    x,y,z = sample_S2.shape

    size_tile =14 

    window_tile_length = int(x/size_tile)
    window_coord = []
    for i in range (0,size_tile):
        window_coord.append(i * window_tile_length)
    window_coord

    reference = rasterio.open(filenames_S2[0])
    reference

    output_meta_class = reference.meta.copy()
    output_meta_class.update(
        {'driver': "GTiff",
        'count': 1})


    output_meta_prob = reference.meta.copy()
    output_meta_prob.update(
        {'driver': "GTiff",
        'count': 10,
        'dtype': rasterio.float32})




    for elem in window_coord:
        with rasterio.open(filenames_S2[0]) as src:
            example = src.read((1), window=Window(0, elem, x, window_tile_length))

        x_ex,y_ex = example.shape
        dat = np.ndarray((x_ex, y_ex))
        dat

        for p in filenames_S2:
            with rasterio.open(p) as src:
                S2 = src.read((1), window=Window(0, elem, x, window_tile_length))
                dat = np.dstack((dat, S2))

        
        dat = np.delete(dat,0,2)
        dat[dat == -9999] = np.nan

        dgm = filenames_dgm[0]
        with rasterio.open(dgm) as src:
            dgm_img = src.read((1), window = Window(0, elem, x, window_tile_length))

        slope = filenames_slope[0]
        with rasterio.open(slope) as src:
            slope_img = src.read((1), window = Window(0, elem, x, window_tile_length))


        dgm_img = np.expand_dims(dgm_img, axis = 2)
        slope_img = np.expand_dims(slope_img, axis = 2)

        dat = np.dstack((dat, dgm_img))
        dat = np.dstack((dat, slope_img))


        q = filenames_S1_VV[0]
        with rasterio.open(q) as src:
            S1_VV = src.read((1,2,3,4,5,6,7,8,9,10,11,12), window = Window(0, elem, x, window_tile_length))

        w = filenames_S1_VH[0]
        with rasterio.open(w) as src:
            S1_VH = src.read((1,2,3,4,5,6,7,8,9,10,11,12), window = Window(0, elem, x, window_tile_length))

        S1_VV = np.swapaxes(S1_VV, 0,2)
        S1_VH = np.swapaxes(S1_VH, 0,2)


        S1_VV = np.swapaxes(S1_VV, 0,1)
        S1_VH = np.swapaxes(S1_VH, 0,1)

        dat = np.dstack((dat, S1_VV))
        dat = np.dstack((dat, S1_VH))



        xs,ys,layer_count = dat.shape
        resh = dat.reshape(-1, layer_count)
        res_prob = fit.predict_proba(resh)
    

        class_count = len(res_prob[0,:])

        layer1 = res_prob[:,0]
        layer1 = layer1.reshape(dat[:, :,1].shape)
        layer1 = np.float32(layer1)


        for i in range(1,class_count):
            layer = res_prob[:,i]
            layer = layer.reshape(dat[:, :,1].shape)
            layer = np.float32(layer)
            #
            layer1 = np.dstack((layer1, layer))

        

        layer_prob = layer1
        layer_prob = reshape_as_raster(layer_prob)
        
    

        out_path = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/out4/'+tile+'/tile_' + str(elem) + '.tif'
        out_path_c = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/out4/'+tile+'/tile_c_' + str(elem) + '.tif'
        with rasterio.open(out_path, "w", **output_meta_prob) as m:
            m.write(layer_prob, window = Window(0, elem, x, window_tile_length))


        with rasterio.open(out_path) as src:
        
            window = get_data_window(src.read(10, masked = True))

            kwargs = src.meta.copy()
            kwargs.update({
                'height': window.height,
                'width': window.width,
                'transform': rasterio.windows.transform(window, src.transform)})

            with rasterio.open(out_path_c, 'w', **kwargs) as dst:
                dst.write(src.read(window=window))


        os.remove(out_path)


        layer_class = np.argmax(layer_prob, axis = 0)
        layer_class = np.expand_dims(layer_class, axis = 0)
    
        

        out_path_class = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/out4/'+tile+'/tile_class_' + str(elem) + '.tif'
        out_path_class_c = '/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/out4/'+tile+'/tile_c_class_' + str(elem) + '.tif'


        with rasterio.open(out_path_class, "w", **output_meta_class) as m:
            m.write(layer_class, window = Window(0, elem, x, window_tile_length))


        with rasterio.open(out_path_class) as src:
        
            window = get_data_window(src.read(1, masked = True))

            kwargs = src.meta.copy()
            kwargs.update({
                'height': window.height,
                'width': window.width,
                'transform': rasterio.windows.transform(window, src.transform)})

            with rasterio.open(out_path_class_c, 'w', **kwargs) as dst:
                dst.write(src.read(window=window))


        os.remove(out_path_class)
        
        del dat, S1_VH, S1_VV
        gc.collect()

/home/nnors/.conda/envs/testenv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but HistGradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/nnors/.conda/envs/testenv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but HistGradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/nnors/.conda/envs/testenv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but HistGradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/nnors/.conda/envs/testenv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but HistGradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/nnors/.conda/envs/testenv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but HistGradientB